<a href="https://colab.research.google.com/github/Carlageo9/Practica_4_Dashboard/blob/main/Practica_4_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tópicos de Indistria I - Práctica 4 Dashboards

**Nombre:** Carla Georgina Sánchez Arreguín

**e-mail:** carla.sanchez2472@alumnos.udg.mx

# MODULES

In [2]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance


# CLASSES

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCIONES

In [4]:
###############################################################################################
# Brownian Motion Trajectory
###############################################################################################
def bm_2d(steps = 1000, speed = 6, xinitPos = 0, yinitPos = 0):
  velocity = Vec2d(speed,0)
  BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos','z_pos','msd','pl'])
  temp_df = pd.DataFrame([{'x_pos':xinitPos,'y_pos':yinitPos,'z_pos':0,'msd':0,'pl':0}])
  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
  for i in range(0,steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity=velocity.rotated(turn_angle)
    temp_df= pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x,'y_pos':BM_2d_df.y_pos[i]+velocity.y,'z_pos':i+1,'msd':0,'pl':0}])
    BM_2d_df = pd.concat([BM_2d_df,temp_df], ignore_index=True)
  return BM_2d_df

In [29]:
###########################################################
### Correlated Random Walks (CRW)
###########################################################
def crw_2d(steps = 1000, speed = 6, xinitPos = 0, yinitPos = 0, fCauchy = 0.4):
  """
  Arguments:
    steps:
    speed:
    s_pos:
    xinitPos:
    yinitPos:
    fCauchy:
  Returns:
    CRW_2d_df:
  """

  #Inicializamos el vector de velocidad
  velocity = Vec2d(speed, 0)

  #Declaramos los DataFrame para guardar la trayectoria
  CRW_2d_df = pd.DataFrame(columns=['x_pos','y_pos','z_pos','msd','pl'])
  temp_df = pd.DataFrame([{'x_pos':xinitPos,'y_pos':yinitPos,'z_pos':0,'msd':0,'pl':0}])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  distribucionDireccion = wrapcauchy(fCauchy,0,scale=1)
  listaDirecciones = distribucionDireccion.rvs(size=steps)

  for i in range(0, steps-1):
    turn_angle = listaDirecciones[i]
    velocity=velocity.rotated(turn_angle)
    temp_df= pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i]+velocity.x,'y_pos':CRW_2d_df.y_pos[i]+velocity.y,'z_pos':i,'msd':0,'pl':0}])
    CRW_2d_df = pd.concat([CRW_2d_df,temp_df], ignore_index=True)

  return CRW_2d_df


In [6]:
def levy_flight(n_steps, speed = 5, xini = 0, yini = 0, CRW_exponent = 0.5, alpha = 1.5, beta = 0, loc = 3.0):
  levy_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos':xini,'y_pos':yini}])
  levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

  velocity = Vec2d(speed,0)
  for i in range(n_steps -1):
    turn_angle = wrapcauchy.rvs(CRW_exponent)
    step_size = levy_stable.rvs(alpha=alpha, beta=beta, loc=loc)
    velocity = velocity.rotated(turn_angle)

    # Calculamos la nueva posición y se agrega al DataFrame de trayectoria
    temp_df = pd.DataFrame([{'x_pos':levy_df.x_pos[i]+(velocity.x*step_size), 'y_pos':levy_df.y_pos[i]+(velocity.y*step_size)}])
    levy_df = pd.concat([levy_df,temp_df], ignore_index=True)
  return levy_df

In [7]:
def trajectoria_Lenght(trayectoria):
  distanciaRetorno = 0.0
  pasos=trayectoria.shape[0]
  Trajectoria = pd.DataFrame(columns=['x_pos','y_pos','z_pos','msd','pl'])
  temp_df = pd.DataFrame([{'x_pos':0,'y_pos':0,'z_pos':0,'msd':0,'pl':0}])
  Trajectoria = pd.concat([Trajectoria, temp_df], ignore_index=True)
  for i in range(1,pasos):
    distanciaRetorno=distanciaRetorno + distancia(trayectoria.x_pos[i-1],trayectoria.y_pos[i-1],trayectoria.x_pos[i],trayectoria.y_pos[i])
    temp_df= pd.DataFrame([{'x_pos':trayectoria.x_pos[i],'y_pos':trayectoria.y_pos[i],'z_pos':i,'msd':trayectoria.msd[i],'pl':distanciaRetorno}])
    Trajectoria = pd.concat([Trajectoria, temp_df], ignore_index=True)
  return Trajectoria

In [22]:
def trajectoria_Mean(trayectoria):
  distanciaRetorno = 0.0
  pasos=trayectoria.shape[0]
  TrajectoryLenghtAC = pd.DataFrame(columns=['x_pos','y_pos','z_pos','msd','pl'])
  temp_df = pd.DataFrame([{'x_pos':0,'y_pos':0,'z_pos':0,'msd':0,'pl':0}])
  TrajectoryLenghtAC = pd.concat([TrajectoryLenghtAC, temp_df], ignore_index=True)
  for i in range(1,pasos-1):
    contadorProm=0
    distanciaRetorno = 0.0
    for ii in range(0,pasos-i):
      contadorProm=contadorProm+1
      #if (ii+i) < (pasos-1) :
      distanciaRetorno=distanciaRetorno + (distancia(trayectoria.x_pos[ii],trayectoria.y_pos[ii],trayectoria.x_pos[ii+i],trayectoria.y_pos[ii+i]))**2
    distanciaRetorno=(distanciaRetorno / contadorProm)
    #print('i: '+str(i)+' ii: '+str(ii)+' distanciaRetrono: '+str(distanciaRetorno))
    temp_df= pd.DataFrame([{'x_pos':trayectoria.x_pos[i-1],'y_pos':trayectoria.y_pos[i-1],'z_pos':trayectoria.z_pos[i-1],'msd':distanciaRetorno,'pl':trayectoria.pl[i-1]}])
    TrajectoryLenghtAC = pd.concat([TrajectoryLenghtAC, temp_df], ignore_index=True)
  return TrajectoryLenghtAC

In [9]:
###########################################################
### Función create_levy_walks proporcionada por Maestro Wario
###########################################################
def create_levy_walks(steps=1000, speed=6, xinitPos = 0, yinitPos = 0, levyExp = 1.5, beta = 1.0, miu = 3.0):
  """
  Arguments:
        n_steps:    int         - Number of trajectory's steps
        speed:      int         - agent's speed (step size)
        s_pos:      [int, int]  - Initial position (x0, y0)
        exponent:   float       - alpha exponent (0 < a < 2]
        beta:       float       - beta [0 < b < 1]
        miu:        int         - Location
  Returns:
        levy_df:  pd.DataFrame  - LF 2D trajectory [(x0, y0), (x1, y1), ..(xn, yn)]
  """

  velocity = Vec2d(speed, 0)
  levy_df = pd.DataFrame(columns=['x_pos', 'y_pos', 'step_length'])
  temporal_df = pd.DataFrame([{
  #    'x_pos': start_position[0],
  #    'y_pos': start_position[1],
      'x_pos': xinitPos,
      'y_pos': yinitPos,
  }])
  levy_df = pd.concat([levy_df, temporal_df], ignore_index=True)
  # definir contador de pasos recorridos
  steps_counter = 0
  # definir contador de posiciones dentro del dataframe
  position = 0
  # iterar mientras el contador de pasos sea menor al numero de pasos por recorrer
  while steps_counter <steps:
    # calcular pasos restantes
    remaining_steps = steps - steps_counter
    steps = levy_stable.rvs(alpha=levyExp, beta=beta)
    steps = math.ceil(steps)
    # obtener el valor absoluto del paso
    steps = abs(steps)
    if steps > remaining_steps:
      steps = remaining_steps
    turn_angle = wrapcauchy.rvs(c=0.5, loc=miu)
    velocity = velocity.rotated(turn_angle)
    for step_index in range(steps):
      # se insertan las nuevas posiciones y la longitud del paso obtenida con levy_stable.rvs
      temporal_df = pd.DataFrame([{
          'x_pos': levy_df.x_pos[position] + (velocity.x),
          'y_pos': levy_df.y_pos[position] + (velocity.y),
      }])
      levy_df = pd.concat([levy_df, temporal_df], ignore_index=True)
      steps_counter += 1
      position += 1

  return levy_df

In [34]:
def LevyFlyPd(steps,speed, miu,levyExponent, Beta, fCauchy,posIniX=0,posIniY=0):
  s_pos = [0,0]
  velocity = Vec2d(speed,0)
  distribucionDireccion = wrapcauchy(0.4,0,scale=1)
  #Se obtienen los giros que se tomaran para la trayectoria
  listaDirecciones = distribucionDireccion.rvs(size=steps)
  #Se crea instancia
  distribucionPasos=levy_stable(alpha=levyExponent, beta=Beta, loc=miu)
  listaCantidadPasos=distribucionPasos.rvs(size=steps)

  pdLevy = pd.DataFrame(columns=['x_pos','y_pos','z_pos','msd','pl'])
  temp3d = pd.DataFrame([{'x_pos':posIniX,'y_pos':posIniY,'z_pos':0,'msd':0,'pl':0}])
  pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
  PasosPendientes=0
  IndicePasos = 0
  IndiceCambiosDireccion = 0

  for i in range(0,steps-1):
    if PasosPendientes < 1 :
      velocity=velocity.rotated(listaDirecciones[IndiceCambiosDireccion])
      IndiceCambiosDireccion=IndiceCambiosDireccion + 1
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x,'y_pos':pdLevy.y_pos[i]+velocity.y,'z_pos':i,'msd':0,'pl':0}])
      pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
      PasosPendientes=int(listaCantidadPasos[IndicePasos])
      IndicePasos=IndicePasos+1
    else :
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x,'y_pos':pdLevy.y_pos[i]+velocity.y,'z_pos':i,'msd':0,'pl':0}])
      pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
      PasosPendientes=PasosPendientes-1

  return pdLevy

In [10]:
###########################################################
### Función para calcular distancia entre 2 puntos
###########################################################
def distancia(x1, y1, x2, y2):
    """
    Calcula la distancia euclidiana entre dos puntos en un espacio bidimensional.

    Args:
        x1 (float): Coordenada x del primer punto.
        y1 (float): Coordenada y del primer punto.
        x2 (float): Coordenada x del segundo punto.
        y2 (float): Coordenada y del segundo punto.

    Returns:
        float: Distancia euclidiana entre los dos puntos.
    """
    #Con la función math.sqrt calculamos la raíz cuadrada, y con ** elevamos a la segunda potencia
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

In [11]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    #Declaramos un vector que va desde la pos_a hasta la pos_b
    vec_ab = np.array([pos_b[0] - pos_a[0], pos_b[1] - pos_a[1]])
    #Se calcula la longitud de ese vector
    norm_ab = np.linalg.norm(vec_ab)

    #Declaramos un vector que va desde la pos_b hasta la pos_c
    vec_bc = np.array([pos_c[0] - pos_b[0], pos_c[1] - pos_b[1]])
    #Se calcula la longitud de ese vector
    norm_bc = np.linalg.norm(vec_bc)

    #Multiplicamos los vectores
    dot_p = np.dot(vec_ab, vec_bc)

    #Se calcula el coseno del ángulo entre los vectores
    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab * norm_bc + np.finfo(float).eps)

    # Angle orientation
    #Calculamos el producto de cruz o producto vectorial
    cross_p = np.cross(vec_ab, vec_bc)
    #Determinamos la orientación del ángulo
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    #La función arccos de NumPy devuelve el arcocoseno de los elementos de la estructura de entrada
    #Calculamos el ángulo en radianes utilizando arccos y se multiplica por la orientación
    theta = np.arccos(np.around(cos_theta,4)) * orient

    return theta

In [12]:
def calculate_step_length(trayectoria):
    trayectoria_con_giro = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_angulo'])
    temp_df = pd.DataFrame([{'x_pos': trayectoria.x_pos[0], 'y_pos': trayectoria.y_pos[0], 'z_angulo': 0}])
    trayectoria_con_giro = pd.concat([trayectoria_con_giro, temp_df], ignore_index=True)

    for i in range(1, trayectoria.shape[0]-1):
        angulo = turning_angle(trayectoria.iloc[i-1], trayectoria.iloc[i], trayectoria.iloc[i+1])
        temp_df = pd.DataFrame([{'x_pos': trayectoria.x_pos[i], 'y_pos': trayectoria.y_pos[i], 'z_angulo': angulo}])
        trayectoria_con_giro = pd.concat([trayectoria_con_giro, temp_df], ignore_index=True)

    trayectoria_con_giro_y_pasos = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_angulo', 'z_steps'])
    temp_df = pd.DataFrame([{'x_pos': trayectoria_con_giro.x_pos[0], 'y_pos': trayectoria_con_giro.y_pos[0], 'z_angulo': 0, 'z_steps': 1}])
    trayectoria_con_giro_y_pasos = pd.concat([trayectoria_con_giro_y_pasos, temp_df], ignore_index=True)

    x_arista = trayectoria_con_giro.x_pos[0]
    y_arista = trayectoria_con_giro.y_pos[0]
    pasos = 1

    for i in range(1, trayectoria_con_giro.shape[0]-1):
        if trayectoria_con_giro.z_angulo[i] == 0:
            pasos += 1
        else:
            temp_df = pd.DataFrame([{'x_pos': trayectoria_con_giro.x_pos[i], 'y_pos': trayectoria_con_giro.y_pos[i],
                                     'z_angulo': trayectoria_con_giro.z_angulo[i], 'z_steps': pasos}])
            trayectoria_con_giro_y_pasos = pd.concat([trayectoria_con_giro_y_pasos, temp_df], ignore_index=True)
            x_arista = trayectoria_con_giro.x_pos[i]
            y_arista = trayectoria_con_giro.y_pos[i]
            pasos = 1

    return trayectoria_con_giro_y_pasos

In [13]:
##################
# Funciones gráficas
#####################
def Grafica_trayectoria_3D(trayectoria,comentario='BM Walk MSD'):
  fig3D = go.Figure(data =[go.Scatter3d(x= trayectoria.x_pos,
                                   y = trayectoria.y_pos,
                                   z = trayectoria.z_pos,
                                   marker=dict(size=2),
                                   line = dict(width=2),
                                   mode ='lines',
                                   name=comentario,
                                   showlegend = True)])
  return fig3D


In [14]:
def Grafica_metricas(trayectoria,metrica=1,comentario='BM Walk MSD'):
  graficaMetrica=go.Figure()
  if metrica == 1 :
    graficaMetrica.add_trace(go.Scatter(
      x=trayectoria.z_pos,
      y=trayectoria.msd,
      marker=dict(size=2),
      line=dict(width=1),
      mode='lines',
      name=comentario,
      showlegend=True
    ))
  else:
    graficaMetrica.add_trace(go.Scatter(
      x=trayectoria.z_pos,
      y=trayectoria.pl,
      marker=dict(size=2),
      line=dict(width=1),
      mode='lines',
      name=comentario,
      showlegend=True
    ))
  return graficaMetrica

# PANEL

In [53]:
##########################
# Declaramos elementos de tablero
##########################
#pn.Row('Parámetros', '3D Trayectoria', 'Métricas')
#Tipo de Random Walk
tipo = pnw.Select(name='Tipo de Random Walk', width=220, value='BM',options=['BM','CRW','Levy'])
tipo
#Número de pasos
steps = pnw.IntSlider(name='Número de Pasos:', width=220, value=500, step=1, start= 0, end= 1000)
steps
#Posición Inicial X-Y
xinitPos = pnw.IntInput(name='Posición Inicial X:', width=100, value=0, step=1, start=0, end=1000)
xinitPos
yinitPos = pnw.IntInput(name='Posición Inicial Y:', width=100, value=0, step=1, start=0, end=1000)
yinitPos
pn.Row(xinitPos, yinitPos)
# Velocidad
speed = pnw.IntSlider(name='Velocidad:', width=220, value=5, step=1, start= 1, end= 20)
# Métricas
metric = pnw.Select(name="Tipo de Métrica", width=220, value='PL', options=['MSD','PL'])
# Para CRW y LF - Coeficiente Cauchy
cauchyExp = pnw.FloatSlider(name='Coeficiente de Cauchy:', width=220, value=0.4, step=0.1, start=0.0, end=1.0)
# Para LF - Exponente Levy (Alpha)
levyExp = pnw.FloatSlider(name='Exponente Levy (alpha):', width=220, value=0.9, step=0.1, start=0.0, end=2.0)
fCauchy = pnw.FloatSlider(name='fCauchy', width=220, value=0.4, step=0.1, start=0.0, end=2.0,visible=False)
miu = pnw.FloatSlider(name='miu', width=220, value=3, step=0.1, start=0.0, end=3.0,visible=False)
beta = pnw.FloatSlider(name='beta:', width=220, value=1.0, step=0.1, start=0.0, end=3.0,visible=False)

#bmt = None
#crwt = None
#lft = None

#fCauchy = 0.4
#miu = 3.0
#beta = 1.0


@pn.depends(tipo, steps, xinitPos, yinitPos, speed, fCauchy, miu, beta, levyExp, cauchyExp)
def figura_3d(tipo, steps, xinitPos, yinitPos, speed, fCauchy, miu, beta, levyExp, cauchyExp):
  if tipo=='BM':
    bmt=bm_2d(steps,speed,xinitPos,yinitPos)
    Grafica_Trayectoria=Grafica_trayectoria_3D(bmt,'BM walk')
    return Grafica_Trayectoria
  elif tipo =='CRW':
    crwt=crw_2d(steps,speed,xinitPos,yinitPos,fCauchy)
    Grafica_Trayectoria=Grafica_trayectoria_3D(crwt,'CRW walk')
    return Grafica_Trayectoria
  elif tipo =='Levy':
    lft=LevyFlyPd(steps,speed,miu,levyExp,beta,fCauchy,xinitPos,yinitPos)
    Grafica_Trayectoria=Grafica_trayectoria_3D(lft,'Levy Walk')
    return Grafica_Trayectoria

@pn.depends(tipo, metric, steps, xinitPos, yinitPos, speed, fCauchy, miu, beta, levyExp, cauchyExp)
def figura_Metrica(tipo, metric, steps, xinitPos, yinitPos, speed, fCauchy, miu, beta, levyExp, cauchyExp):
  if tipo=='BM':
    if metric == 'PL':
      bmt=bm_2d(steps,speed,xinitPos,yinitPos)
      grafica = Grafica_metricas(trajectoria_Lenght(bmt),2,'PL BM')
    else:
      grafica =  Grafica_metricas(trajectoria_Mean(bmt),1,'MSD BM')
    return grafica
  elif tipo =='CRW':
    if metric == 'PL':

      crwt=crw_2d(steps,speed,xinitPos,yinitPos,fCauchy)
      grafica =  Grafica_metricas(trajectoria_Lenght(crwt),2,'PL CRW')
    else:
      grafica =  Grafica_metricas(trajectoria_Mean(crwt),1,'MSD CRW')
    return grafica
  elif tipo =='Levy':
    if metric == 'PL':
      grafica =  Grafica_metricas(trajectoria_Lenght(lft),2,'PL Levy')
    else:
      grafica =  Grafica_metricas(trajectoria_Mean(lft),1,'MSD Levy')
    return grafica


pn.Row(pn.Column(tipo, steps, pn.Row(xinitPos, yinitPos), speed, metric, cauchyExp, levyExp),figura_3d,figura_Metrica)

Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 478, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 431, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 408, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 471, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 442, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 469, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 451, in _schedule_change
 self._change_event(doc)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 447, in _change_event
 self._process_events(events)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 385, in _process_events
 self.param.update(**self_events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2278, in update
 restore = self_._update(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2318, in _update
 self_._batch_call_watchers()
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2509, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2471, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 865, in _replace_pane
 new_object = self.eval(self.object)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 824, in eval
 return eval_function_with_deps(function)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 162, in eval_function_with_deps
 return function(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/param/depends.py", line 41, in _depends
 return func(*args, **kw)
 File "<ipython-input-53-cb7e4e73c6c8>", line 60, in figura_Metrica
 grafica = Grafica_metricas(trajectoria_Mean(bmt),1,'MSD BM')
UnboundLocalError: local variable 'bmt' referenced before assignment

Row
    [0] Column
        [0] Select(name='Tipo de Random Walk', options=['BM', 'CRW', 'Levy'], value='BM', width=220)
        [1] IntSlider(end=1000, name='Número de Pasos:', value=500, width=220)
        [2] Row
            [0] IntInput(end=1000, name='Posición Inicial X:', start=0, width=100)
            [1] IntInput(end=1000, name='Posición Inicial Y:', start=0, width=100)
        [3] IntSlider(end=20, name='Velocidad:', start=1, value=5, width=220)
        [4] Select(name='Tipo de Métrica', options=['MSD', 'PL'], value='PL', width=220)
        [5] FloatSlider(name='Coeficiente de Cauchy:', value=0.4, width=220)
        [6] FloatSlider(end=2.0, name='Exponente Levy (alpha):', value=0.9, width=220)
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)
    [2] ParamFunction(function, _pane=Plotly, defer_load=False)

2024-03-11 05:53:49,273 ERROR: panel.reactive - Callback failed for object named "Tipo de Métrica" changing property {'value': 'MSD'} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2278, in update
    restore = self_._update(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2318, in _update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2509, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2471, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 865, in _replace_pane
    new_object = self.eval(self.object)
  File "/